# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_ids, target_ids = [], []
    source_split = source_text.split("\n")
    for s in source_split:
        ids = [source_vocab_to_int[word] for word in s.split()]
        source_ids.append(ids)
    target_split = target_text.split("\n")
    for s in target_split:
        ids = [target_vocab_to_int[word] for word in s.split()]
        ids.append(target_vocab_to_int['<EOS>'])
        target_ids.append(ids)

    return (source_ids, target_ids)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    _input = tf.placeholder(tf.int32, [None, None], name="input")
    _targets = tf.placeholder(tf.int32, [None, None], name="targets")
    _lr = tf.placeholder(tf.float32, name="learning_rate")
    _keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    _target_sequence_length = tf.placeholder(tf.int32, (None,), name="target_sequence_length")
    _max_target_sequence_length = tf.reduce_max(_target_sequence_length, name="max_target_len")
    _source_sequence_length = tf.placeholder(tf.int32, (None,), name="source_sequence_length")
    return (
        _input, _targets, _lr, _keep_prob, _target_sequence_length, 
        _max_target_sequence_length, _source_sequence_length)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    # worth mentioning that I had a hard time understanding strided_slice, but basically
    # you are drawing a rectangle with two coordinates, [0,0] is the top left corner
    # [batch_size, -1] is the bottom right corner, and [1,1] just means your stride
    # so (in this case, keep everything)
    ending = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    # this basically creates a rank 2 tensor of '<GO'>, that is batch_size x 1, like a vertical vector
    # it then pre-pends it to the ending tensor which is batch_size x (len(target[1]) - 1)
    # and it does this along the column axis (axis = 1)
    decoder_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return decoder_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # Taking the input, you use the vocabulary size, and encoding_embedding size as two of
    # the i don't know 3?? dimensions, this needs futher study
    enc_embedding = tf.contrib.layers.embed_sequence(
        rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # encoder
    def make_cell(rnn_size):
        # construct our cell and initialize
        # made of LSTM cells
        enc_cell = tf.contrib.rnn.DropoutWrapper(
            tf.contrib.rnn.LSTMCell(
                rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2)),
            input_keep_prob=keep_prob)
        
        return enc_cell # this is just a single layer of our encoder
    
    # now to make the full multi_rnn_cell of num_layer encoding cells
    enc_cell = tf.contrib.rnn.MultiRNNCell(
        [make_cell(rnn_size) for _ in range(num_layers)])
    # print(enc_cell)
    
    # lets embed the input
    enc_output, enc_state = tf.nn.dynamic_rnn(
        enc_cell, enc_embedding, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return (enc_output, enc_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    # helper for the training process.  used by BasicDecoder to read inputs.
    # whatever this means :P
    # print(max_summary_length.shape, max_summary_length.dtype)
    training_helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=dec_embed_input, sequence_length=target_sequence_length, time_major=False)
    # basic decoder
    # ?? get the max target sequence length
    #max_target_sequence_length = tf.reduce_max(target_sequence_length)
    # wrapping a dropout for keep probability
    # print(dec_cell)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob)
    # print(dec_cell)
    # make the training decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, training_helper, encoder_state, output_layer)
    # perform dynamic decoding using the decoder
    x = tf.contrib.seq2seq.dynamic_decode(
        training_decoder, impute_finished=True, maximum_iterations=max_summary_length)
    basic_decoder_output = x[0]

    return basic_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # create a 1d tensor of <GO> codes as our start tokens
    start_tokens = tf.tile(
        tf.constant([start_of_sequence_id], dtype=tf.int32),
        [batch_size], name="start_tokens")
    
    # Helper for the inference process
    # This is an interesting function, it needs a vector of start_sequences, and a scalar
    # of the end_of_sequence
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, start_tokens, end_of_sequence_id)
    

    # adding dropout wrapper to the decode cell
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob)
    
    # Basic decoder    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, inference_helper, encoder_state, output_layer)
    
    # Perform dynamic decoding using the decoder
    x = tf.contrib.seq2seq.dynamic_decode(
        inference_decoder, impute_finished=True, maximum_iterations=max_target_sequence_length)
    
    basic_decoder_output = x[0]
    
    return basic_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder embedding
    dec_embeddings = tf.Variable(tf.random_uniform(
        [target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        # construct our cell and initialize
        # made of LSTM cells
        dec_cell = tf.contrib.rnn.DropoutWrapper(
            tf.contrib.rnn.LSTMCell(
                rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2)),
            input_keep_prob=keep_prob)
        
        return dec_cell # this is just a single layer of our encoder
    
    # Stack layers
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])  
    
    # 3. Dense layer to translate decoders outputs at each time step into a choice from the
    # target vocabulary
    output_layer = Dense(
        target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    
    # 4. Set up training decoder
    with tf.variable_scope("decode"):
        train = decoding_layer_train(
            encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length,
            output_layer, keep_prob)
    
    # 5. Set up inference decoder
    with tf.variable_scope("decode", reuse=True):
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']
        infer = decoding_layer_infer(
            encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
            max_target_sequence_length, target_vocab_size, output_layer, batch_size, keep_prob)
    
    return (train, infer)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Encode the input
    _, enc_state = encoding_layer(
        input_data, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size,
        enc_embedding_size)
    # 2. Process target data
    dec_input = process_decoder_input(
        target_data, target_vocab_to_int, batch_size)
    
    # 3. Decode the encoded input using the decoding layer
    train, infer = decoding_layer(
        dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size,
        num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    
    return train, infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [22]:
# Number of Epochs
epochs = 20
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 150
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.75
display_step = 50

### Build the Graph
Build the graph using the neural network you implemented.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [25]:
from tqdm import tqdm
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in tqdm(range(epochs)):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch   0 Batch   50/538 - Train Accuracy: 0.3299, Validation Accuracy: 0.3906, Loss: 2.9994
Epoch   0 Batch  100/538 - Train Accuracy: 0.4225, Validation Accuracy: 0.4751, Loss: 2.5386
Epoch   0 Batch  150/538 - Train Accuracy: 0.4643, Validation Accuracy: 0.5051, Loss: 2.2838
Epoch   0 Batch  200/538 - Train Accuracy: 0.4641, Validation Accuracy: 0.5140, Loss: 2.0890
Epoch   0 Batch  250/538 - Train Accuracy: 0.4637, Validation Accuracy: 0.5238, Loss: 1.9422
Epoch   0 Batch  300/538 - Train Accuracy: 0.4712, Validation Accuracy: 0.4877, Loss: 1.6597
Epoch   0 Batch  350/538 - Train Accuracy: 0.4950, Validation Accuracy: 0.5137, Loss: 1.5511
Epoch   0 Batch  400/538 - Train Accuracy: 0.5086, Validation Accuracy: 0.5302, Loss: 1.3874
Epoch   0 Batch  450/538 - Train Accuracy: 0.5372, Validation Accuracy: 0.5421, Loss: 1.2669
Epoch   0 Batch  500/538 - Train Accuracy: 0.5259, Validation Accuracy: 0.5336, Loss: 1.1211


  5%|▌         | 1/20 [00:55<17:39, 55.76s/it]

Epoch   1 Batch   50/538 - Train Accuracy: 0.5236, Validation Accuracy: 0.5513, Loss: 1.0481
Epoch   1 Batch  100/538 - Train Accuracy: 0.5350, Validation Accuracy: 0.5616, Loss: 0.9892
Epoch   1 Batch  150/538 - Train Accuracy: 0.5516, Validation Accuracy: 0.5497, Loss: 0.9419
Epoch   1 Batch  200/538 - Train Accuracy: 0.5574, Validation Accuracy: 0.5614, Loss: 0.8972
Epoch   1 Batch  250/538 - Train Accuracy: 0.5514, Validation Accuracy: 0.5723, Loss: 0.8655
Epoch   1 Batch  300/538 - Train Accuracy: 0.5844, Validation Accuracy: 0.5760, Loss: 0.8136
Epoch   1 Batch  350/538 - Train Accuracy: 0.5755, Validation Accuracy: 0.5653, Loss: 0.8178
Epoch   1 Batch  400/538 - Train Accuracy: 0.5701, Validation Accuracy: 0.5877, Loss: 0.7838
Epoch   1 Batch  450/538 - Train Accuracy: 0.6068, Validation Accuracy: 0.6014, Loss: 0.7841
Epoch   1 Batch  500/538 - Train Accuracy: 0.6087, Validation Accuracy: 0.6065, Loss: 0.6852


 10%|█         | 2/20 [01:51<16:42, 55.68s/it]

...

Epoch  18 Batch   50/538 - Train Accuracy: 0.9250, Validation Accuracy: 0.9387, Loss: 0.0632
Epoch  18 Batch  100/538 - Train Accuracy: 0.9463, Validation Accuracy: 0.9382, Loss: 0.0515
Epoch  18 Batch  150/538 - Train Accuracy: 0.9484, Validation Accuracy: 0.9364, Loss: 0.0600
Epoch  18 Batch  200/538 - Train Accuracy: 0.9430, Validation Accuracy: 0.9370, Loss: 0.0497
Epoch  18 Batch  250/538 - Train Accuracy: 0.9543, Validation Accuracy: 0.9318, Loss: 0.0606
Epoch  18 Batch  300/538 - Train Accuracy: 0.9382, Validation Accuracy: 0.9455, Loss: 0.0556
Epoch  18 Batch  350/538 - Train Accuracy: 0.9507, Validation Accuracy: 0.9384, Loss: 0.0638
Epoch  18 Batch  400/538 - Train Accuracy: 0.9418, Validation Accuracy: 0.9405, Loss: 0.0575
Epoch  18 Batch  450/538 - Train Accuracy: 0.9228, Validation Accuracy: 0.9318, Loss: 0.0674
Epoch  18 Batch  500/538 - Train Accuracy: 0.9602, Validation Accuracy: 0.9366, Loss: 0.0445


 95%|█████████▌| 19/20 [17:32<00:55, 55.01s/it]

Epoch  19 Batch   50/538 - Train Accuracy: 0.9436, Validation Accuracy: 0.9395, Loss: 0.0508
Epoch  19 Batch  100/538 - Train Accuracy: 0.9582, Validation Accuracy: 0.9498, Loss: 0.0474
Epoch  19 Batch  150/538 - Train Accuracy: 0.9490, Validation Accuracy: 0.9432, Loss: 0.0497
Epoch  19 Batch  200/538 - Train Accuracy: 0.9482, Validation Accuracy: 0.9428, Loss: 0.0442
Epoch  19 Batch  250/538 - Train Accuracy: 0.9563, Validation Accuracy: 0.9368, Loss: 0.0518
Epoch  19 Batch  300/538 - Train Accuracy: 0.9522, Validation Accuracy: 0.9464, Loss: 0.0521
Epoch  19 Batch  350/538 - Train Accuracy: 0.9501, Validation Accuracy: 0.9426, Loss: 0.0596
Epoch  19 Batch  400/538 - Train Accuracy: 0.9420, Validation Accuracy: 0.9407, Loss: 0.0536
Epoch  19 Batch  450/538 - Train Accuracy: 0.9252, Validation Accuracy: 0.9439, Loss: 0.0621
Epoch  19 Batch  500/538 - Train Accuracy: 0.9640, Validation Accuracy: 0.9350, Loss: 0.0368


100%|██████████| 20/20 [18:27<00:00, 54.91s/it]


Model Trained and Saved


### Hyperparameter selections and results

|embed_input|embed_output|batch_size| rnn_size|keep_prob|    lr|epochs|training_time| train_acc| val_acc|  loss|
|--------------------------------------------------------------------------------------------------------------|
|         30|          30|       128|       50|     0.50|0.0010|    10|          ?  |    0.6684|    0.7248|0.3571|
|         15|          15|       256|      100|     0.50|0.0010|    10|        09:10|    0.7305|    0.7228|0.3687|
|         15|          15|       512|      100|     0.75|0.0010|    10|        04:56|    0.6604|    0.6671|0.4456|
|         15|          15|       512|      100|     0.75|0.0005|    10|        04:53|    0.6127|    0.6345|0.6592|
|         15|          15|       512|      100|     0.75|0.0005|    30|        14:50|    0.8104|    0.7900|0.2596|
|         15|          15|       256|      100|     0.75|0.0010|    20|        18:02|    0.8766|    0.8699|0.1053|
|         15|          15|       256|      150|     0.75|0.0010|    20|        18:27|__0.9640__|__0.9350__|0.0368|

### Analysis

It appears that the biggest gains in terms of accuracy came from increasing the __rnn_size__, keep_prob, and epochs.
The __rnn_size__ seems especially important.  Increasing the batch size seems to speed training, but it may cost accuracy.

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [28]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence = sentence.lower()
    seq = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split()]
    
    return seq


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [29]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [71, 156, 125, 129, 178, 75, 43]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [357, 228, 105, 331, 331, 198, 1]
  French Words: est a vu aux aux . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.